<a href="https://colab.research.google.com/github/Voolshara/Nti_AI_academy_2020/blob/main/BERT_MULTI_15E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers==3.0.0

     |████████████████████████████████| 757kB 9.7MB/s 
     |████████████████████████████████| 3.0MB 16.4MB/s 
     |████████████████████████████████| 1.1MB 46.5MB/s 
     |████████████████████████████████| 890kB 56.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a0883772747083f14ad52ce541eb409a5fd2f1279b151d96adb4fef01b950251
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl

--2021-01-09 17:12:13--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2930172 (2.8M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   2.79M  --.-KB/s    in 0.07s   

2021-01-09 17:12:13 (37.7 MB/s) - ‘train.jsonl’ saved [2930172/2930172]



In [ ]:
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl

--2021-01-09 17:12:13--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 566932 (554K) [text/plain]
Saving to: ‘val.jsonl’

val.jsonl           100%[===================>] 553.64K  --.-KB/s    in 0.03s   

2021-01-09 17:12:14 (17.6 MB/s) - ‘val.jsonl’ saved [566932/566932]



In [ ]:
!wget  https://raw.githubusercontent.com/Voolshara/DS_NTI/main/test.jsonl

--2021-01-09 17:12:14--  https://raw.githubusercontent.com/Voolshara/DS_NTI/main/test.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1739290 (1.7M) [text/plain]
Saving to: ‘test.jsonl’

test.jsonl          100%[===================>]   1.66M  --.-KB/s    in 0.09s   

2021-01-09 17:12:14 (17.5 MB/s) - ‘test.jsonl’ saved [1739290/1739290]



In [ ]:
!pip install jsonlines

In [ ]:
import json
import jsonlines
import codecs
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def get_Data(data_json_file1, data_json_file2):
    X, Y = [], []
    with open(data_json_file1, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                  if len(text) <= 512:
                    text_n = text
                  else:
                    text_n = text[-542:]   
                  qw = query[:-1] if query[-1]=="." else query
                  txt = a["text"][:-1] if a["text"][-1] == "." else a["text"] 
                  X.append("[CLS]" +text_n+ "[SEP] " +qw+ " " +txt)
                  if a['label'] == 1:
                    Y.append([0, 1])
                  else:
                    Y.append([1, 0])  
                  #Y.append(a["label"]) 

    with open(data_json_file2, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                  if len(text) <= 512:
                    text_n = text
                  else:
                    text_n = text[-500:]   
                  qw = query[:-1] if query[-1]=="." else query
                  txt = a["text"][:-1] if a["text"][-1] == "." else a["text"] 
                  X.append("[CLS]" +text_n+ "[SEP] " +qw+ " " +txt)
                  if a['label'] == 1:
                    Y.append([0, 1])
                  else:
                    Y.append([1, 0])  
                  #Y.append(a["label"])                          
    return X, Y

In [ ]:
Sentences, Labels = get_Data("/content/train.jsonl", "/content/val.jsonl")

In [ ]:
print(Sentences[100])
print(len(Sentences), len(Sentences) == len(Labels))

[CLS]участие в 11 матчах. (9) В среднем он отражает 90,5 процента бросков по своим воротам и пропускает по 3,1 шайбы за игру. (10) Один матч россиянин отстоял "на ноль". (11) Из других матчей, сыгранных в ночь на 8 ноября, стоит отметить победу "Вашингтон Кэпиталс" на "Филадельфией Флайерс" (3:2 в овертайме). (12) Нападающий "Вашингтона" Александр Семин забросил шайбу, а его одноклубник Александр Овечкин отдал голевой пас. (13) Один из голов "Филадельфии" на счету Николая Жердева Вратарь "Флайерс" Сергей Бобровский отразил 36 из 39 бросков."[SEP] Как Николай Хабибулин принес своей команде победу восьмого ноября в Чикаго? Он поймал все шайбы противника
14185 True


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
R = Sentences[100]
# Print the original sentence.
print(' Original: ', R)

# Print the sentence split into tokens.
#print('Tokenized: ', tokenizer(R, return_tensors))
print('Tokenized: ', tokenizer.tokenize(R, add_special_tokens=True))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(R)))

Keyword arguments {'add_special_tokens': True} not recognized.


 Original:  [CLS]участие в 11 матчах. (9) В среднем он отражает 90,5 процента бросков по своим воротам и пропускает по 3,1 шайбы за игру. (10) Один матч россиянин отстоял "на ноль". (11) Из других матчей, сыгранных в ночь на 8 ноября, стоит отметить победу "Вашингтон Кэпиталс" на "Филадельфией Флайерс" (3:2 в овертайме). (12) Нападающий "Вашингтона" Александр Семин забросил шайбу, а его одноклубник Александр Овечкин отдал голевой пас. (13) Один из голов "Филадельфии" на счету Николая Жердева Вратарь "Флайерс" Сергей Бобровский отразил 36 из 39 бросков."[SEP] Как Николай Хабибулин принес своей команде победу восьмого ноября в Чикаго? Он поймал все шайбы противника
Tokenized:  ['[CLS]', 'участие', 'в', '11', 'матчах', '.', '(', '9', ')', 'в', 'среднем', 'он', 'от', '##ра', '##жает', '90', ',', '5', 'про', '##цент', '##а', 'бр', '##ос', '##ков', 'по', 'своим', 'ворота', '##м', 'и', 'про', '##пуска', '##ет', 'по', '3', ',', '1', 'ша', '##иб', '##ы', 'за', 'игру', '.', '(', '10', ')', 'один

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in Sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        max_length = 468,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', Sentences[0])
print('Token IDs:', input_ids[0])

Original:  [CLS]к они, слипшиеся, неподъёмно и лежали. (7) Да и не ради причёски приходил парень к Людочке. (8) Как только её руки становились занятыми ножницами и расчёской, Артемка начинал хватать её за разные места. (9) Людочка сначала увёртывалась от хватких рук Артемки, а когда не помогло, стукнула его машинкой по голове и пробила до крови, пришлось лить йод на голову «ухажористого человека». (10) Артемка заулюлюкал и со свистом стал ловить воздух. (11) С тех пор «домогания свои хулиганские прекратил», более того, шпане повелел Людочку не трогать.[SEP] Где бегала шпана? В парке
Token IDs: tensor([   101,    551,  14274,    117,  52399,  30637,  78152,    117,  10375,
         53204,  10746,  30318,  12579,  10636,    549,  94693,  48873,    119,
           113,    128,    114,  10405,    549,  10375,  18701,  10913,  24373,
         10913,  82005,  66457,  19557,    551,    552,  10593,  17961,  19347,
           119,    113,    129,    114,  10949,  13721,  75431,  50365,  66345,

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

12,766 training samples
1,419 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
epochs = 15

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 15 ========
Training...
  Batch    40  of  1,596.    Elapsed: 0:00:18.
  Batch    80  of  1,596.    Elapsed: 0:00:36.
  Batch   120  of  1,596.    Elapsed: 0:00:54.
  Batch   160  of  1,596.    Elapsed: 0:01:12.
  Batch   200  of  1,596.    Elapsed: 0:01:30.
  Batch   240  of  1,596.    Elapsed: 0:01:48.
  Batch   280  of  1,596.    Elapsed: 0:02:06.
  Batch   320  of  1,596.    Elapsed: 0:02:24.
  Batch   360  of  1,596.    Elapsed: 0:02:42.
  Batch   400  of  1,596.    Elapsed: 0:03:00.
  Batch   440  of  1,596.    Elapsed: 0:03:18.
  Batch   480  of  1,596.    Elapsed: 0:03:36.
  Batch   520  of  1,596.    Elapsed: 0:03:54.
  Batch   560  of  1,596.    Elapsed: 0:04:12.
  Batch   600  of  1,596.    Elapsed: 0:04:29.
  Batch   640  of  1,596.    Elapsed: 0:04:47.
  Batch   680  of  1,596.    Elapsed: 0:05:05.
  Batch   720  of  1,596.    Elapsed: 0:05:23.
  Batch   760  of  1,596.    Elapsed: 0:05:41.
  Batch   800  of  1,596.    Elapsed: 0:05:59.
  Batch   840  o

In [ ]:
def get_Data_Pred(data_json_file1):
    X, Y = [], []
    with open(data_json_file1, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                  if len(text) <= 512:
                    text_n = text
                  else:
                    text_n = text[-542:]   
                  qw = query[:-1] if query[-1]=="." else query
                  txt = a["text"][:-1] if a["text"][-1] == "." else a["text"] 
                  X.append("[CLS]" +text_n+ "[SEP] " +qw+ " " +txt)                         
    return X

In [ ]:
Pred = get_Data_Pred("/content/test.jsonl")

In [ ]:
torch.cuda.set_device(0)

In [ ]:
model = model.to("cuda:0")

In [ ]:
model = torch.nn.DataParallel(model.cuda(), device_ids=[0])

In [ ]:
BERT = []
for el in Pred:
  try:
    token = tokenizer(el, return_tensors="pt")
    out = model(**token)
    i = list(map(float, str(out)[11:-48].split(',')))
    BERT.append(i)
  except:
    print(el)  

In [ ]:
file = open("out.txt", "w")
for el in BERT:
  file.write(str(el) + '\n')
file.close()  

In [ ]:
A = []
for i in BERT:
   if i[1] > 1:
      A.append(1)
   else:
      A.append(0)  

In [ ]:
from pprint import pprint
pprint(BERT[:8])

[[1.2891, 1.096],
 [1.2856, 1.093],
 [1.0879, -0.791],
 [1.1477, -0.848],
 [1.4167, -1.086],
 [1.2944, -0.98],
 [1.2768, -0.965],
 [0.7381, 0.676]]


In [ ]:
def raz(r):
  if r == 1:
    return 0
  return 1  

In [ ]:
All_Dict = []
i = 0
with jsonlines.open('test.jsonl') as reader:
    for jsn in reader:
        Main_Dict = dict()
        passage = jsn["passage"]
        Main_Dict['idx'] = jsn["idx"]
        Passage = dict()
        Passage["text"] = str(passage["text"])
        out2_arr = []
        for ques in passage["questions"]:
            qu = dict()
            qu["question"] = str(ques["question"])
            answers = ques["answers"]
            out3_arr = []
            for el in answers:
                out3 = dict()
                out3["idx"] = el["idx"]
                out3["text"] = str(el["text"])
                out3["label"] = raz(A[i])
                #out3["label"] = A[i]
                out3_arr.append(out3)
                i += 1
            qu["answers"] = out3_arr
            qu["idx"] = ques["idx"]
            out2_arr.append(qu)
        Passage["questions"] = out2_arr
        Main_Dict["passage"] = Passage
        jn = str(Main_Dict)
        #print(jn)
        All_Dict.append(Main_Dict)
with jsonlines.open('output.jsonl', 'w') as writer:
    writer.write_all(All_Dict)